## 準備
フォルダに、`gait.mp4`をアップロードしましょう。

## ライブラリ類のインポートなど

In [ ]:
import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

def imshow(img):  # Colab用imshow関数として
  if img.ndim == 3:
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    display(Image.fromarray(img))
  else:
    display(Image.fromarray(img))

## 繰り返し最小化による速度ベクトルの導出

1. 準備

In [ ]:
# 動画の読み込み
cap = cv2.VideoCapture('gait.mp4')

frame_first = 0  # 最初のフレーム
frame_second = 1 # 移動先のフレーム

# 動画を画像列として格納
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # グレースケール、[0,1]の値として輝度値を格納した画像にする。I_0 -> I_1
    if frame_count == frame_first:
      frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
      I_0 = frame.astype(float) / 255.0
    if frame_count == frame_second:
      frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
      I_1 = frame.astype(float) / 255.0

    frame_count = frame_count + 1
cap.release()

# 中心差分近似によるI_xの導出
kernel = 0.5 * np.array([[0, 0, 0], [-1, 0, 1], [0, 0, 0]])
I_x = cv2.filter2D(I_0, -1, kernel)

# 中心差分近似によるI_yの導出
kernel = 0.5 * np.array([[0, -1, 0], [0, 0, 0], [0, 1, 0]])
I_y = cv2.filter2D(I_0, -1, kernel)

# 時間方向の微分 I_t
I_t = I_1 - I_0

# 表示
print("入力 I_0, I_1")
imshow(np.hstack([(I_0*255).astype(np.uint8),(I_1*255).astype(np.uint8)]))
print("微分 I_x, I_y, I_t")
imshow(np.hstack([(I_x*128 + 128).astype(np.uint8),(I_y*128 + 128).astype(np.uint8),(I_t*128 + 128).astype(np.uint8)]))

2. 繰り返し最適化

In [ ]:
# 速度ベクトル (u,v) の初期化
u = np.zeros_like(I_0)
v = np.zeros_like(I_0)
alpha = 0.3

for i in range(10):
  print(str(i) + "-th iteration")

  kernel = 0.25 * np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
  u_bar = cv2.filter2D(u, -1, kernel)
  v_bar = cv2.filter2D(v, -1, kernel)

  u = ((I_y * I_y + alpha) * u_bar - I_x * I_y * v_bar - I_x * I_t) / (I_x * I_x + I_y * I_y + alpha)
  v = ((I_x * I_x + alpha) * v_bar - I_x * I_y * u_bar - I_y * I_t) / (I_x * I_x + I_y * I_y + alpha)

  print("Optical flow")
  # この表示方法だと、128画素以上移動したと推定された場合にオーバーフローするので注意
  imshow(np.hstack([(u * 128 + 128).astype(np.uint8),(v * 128 + 128).astype(np.uint8)]))

## ちなみに...

OpenCVにも密なオプティカルフロー推定手法が（もちろん）実装されています。

In [ ]:
uv = cv2.calcOpticalFlowFarneback((I_0*255).astype(np.uint8), (I_1*255).astype(np.uint8), None, 0.5, 1, 5, 5, 5, 1.1, 0)

u = uv[..., 0]
v = uv[..., 1]

print("Optical flow")
# この表示方法だと、128画素以上移動したと推定された場合にオーバーフローするので注意
imshow(np.hstack([(u * 128 + 128).astype(np.uint8),(v * 128 + 128).astype(np.uint8)]))